In [1]:
%load_ext autoreload
%autoreload 2

# Planning

## Small Grid World
Below is the definition of a small grid world as described in Silver's lecture.
  * Undiscounted episodic MDP ($\gamma = 1$)
  * 4x4 grid
  * 2 terminal states: (0, 0) and (3, 3)
  * Actions leading out of the grid leave the state unchanged
  * Reward is -1 until the terminal state is reached
  * Agent follows uniform random policy

$$
\pi(n \vert \cdot) = \pi(e \vert \cdot) = \pi(s \vert \cdot) = \pi(w \vert \cdot) = \frac{1}{4}
$$

## Policy Evaluation

### Belmann's Equation

$$
\begin{align}
v_{\pi}(s) &= \mathbb E_{\pi, P} \left[ r(s, a) + \gamma P(s' \vert s, a) v(s') \right] \\
&= \sum_{a \in \mathcal A} \pi(a \vert s) \left\{ r(s, a) + \gamma \sum_{s' \in \mathcal S} P(s' \vert s, a) v(s') \right\}
\end{align}
$$

Vectorized form of the above equation is -
$$
\mathbf v_{\pi} = \mathbf r + \gamma P_{\pi} \mathbf v_{\pi}
$$

Two ways to solve this - 
  (i) Vectorized Form
  (ii) Dynamic Programming

### Vectorized Form
$$
\mathbf v_{\pi} = (I - \gamma P_{\pi})^{-1} \mathbf r
$$

### Dynamic Programming
$$
v_{k+1}(s) = \sum_{a \in \mathcal A} \pi(a \vert s) \left\{ r(s, a) + \gamma \sum_{s' \in \mathcal S} P(s' \vert s, a) v_k(s') \right\}
$$

In [2]:
import numpy as np

In [3]:
# Knowing the full MDP, I can calculate the state values for this specific problem.
n_iters = 2048
values = np.zeros((4, 4), dtype=np.float32)
values_next = np.zeros((4, 4), dtype=np.float32)

up = lambda r, c: (max(0, r - 1), c)
down = lambda r, c: (min(3, r + 1), c)
left = lambda r, c: (r, max(0, c - 1))
right = lambda r, c: (r, min(3, c + 1))

for i in range(n_iters):
    for r in range(4):
        for c in range(4):
            if (r == 0 and c == 0) or (r == 3 and c == 3):
                continue
            values_next[r, c] = -1 + 0.25 * (
                values[up(r, c)]
                + values[down(r, c)]
                + values[left(r, c)]
                + values[right(r, c)]
            )
    if np.allclose(values_next, values):
        print(f"Got convergence after {i} iters.")
        break
    values = values_next
    values_next = np.zeros((4, 4), dtype=np.float32)

np.rint(values)

Got convergence after 158 iters.


array([[  0., -14., -20., -22.],
       [-14., -18., -20., -20.],
       [-20., -20., -18., -14.],
       [-22., -20., -14.,   0.]], dtype=float32)

In [4]:
from functools import partial
from small_grid_world import (
    Policy,
    SmallGridWorld,
    State,
    Grid,
    Action,
    argmax,
)

MAX_ITERS = 1000

In [5]:
def _calc_v_next(mdp: SmallGridWorld, v: Grid, pi: Policy, s: State) -> float:
    r = mdp.reward
    p = mdp.prob
    γ = mdp.gamma

    v_ = 0.0
    for a in mdp.actions():
        q = r(s, a) + γ * sum(p(s_, given=(s, a)) * v[s_] for s_ in mdp.states())
        v_ += pi(a, given=s) * q
    return v_


def calc_svals(mdp: SmallGridWorld, pi: Policy) -> Grid:
    v = Grid(4, random=True)
    v_next = Grid(4)
    i = 0
    while i < MAX_ITERS and not v.close(v_next):
        v.copy(v_next)
        v_next.clear()
        for s in mdp.states():
            v_next[s] = _calc_v_next(mdp, v, pi, s)
        i += 1
    print(f"Converged after {i} iterations.")
    v.round()
    return v

In [6]:
def generate_uniform_random_policy(mdp):
    def policy(a, given):
        return 0 if mdp.is_terminal(given) else 0.25

    return policy

In [7]:
small_grid = SmallGridWorld()
v = calc_svals(small_grid, generate_uniform_random_policy(small_grid))
v

Converged after 159 iterations.


array([[  0., -14., -20., -22.],
       [-14., -18., -20., -20.],
       [-20., -20., -18., -14.],
       [-22., -20., -14.,   0.]], dtype=float32)

In [8]:
def qval(mdp: SmallGridWorld, v: Grid, s: State, a: Action) -> float:
    r = mdp.reward
    p = mdp.prob
    γ = mdp.gamma

    return r(s, a) + γ * sum(p(s_, given=(s, a)) * v[s_] for s_ in mdp.states())


def generate_greedy_policy(mdp: SmallGridWorld, v: Grid) -> Policy:
    def policy(action: Action, given: State, dbg=False) -> float:
        s = given
        q_s = partial(qval, mdp, v, s)

        if mdp.is_terminal(s):
            return 0.0

        best_actions = argmax(mdp.actions(), key=lambda a: q_s(a))
        best_action = best_actions[0]
        if dbg and np.array_equal(action, best_action):
            if len(best_actions) > 1:
                best_actions_str = ", ".join(str(a) for a in best_actions)
                print(
                    f"\tDEBUG: pi(action={action}, state={s}): Got {len(best_actions)} best actions: {best_actions_str}, choosing {best_action}"
                )

        if action == best_action:
            return 1.0

        return 0.0

    return policy

In [9]:
v

array([[  0., -14., -20., -22.],
       [-14., -18., -20., -20.],
       [-20., -20., -18., -14.],
       [-22., -20., -14.,   0.]], dtype=float32)

In [10]:
gp = generate_greedy_policy(small_grid, v)
for s in small_grid.states():
    for a in small_grid.actions():
        prob = gp(a, s, dbg=True)
        if prob == 1.0:
            print(f"π({s}) = {a}")

π((0, 1)) = ←
π((0, 2)) = ←
	DEBUG: pi(action=↓, state=(0, 3)): Got 2 best actions: ↓, ←, choosing ↓
π((0, 3)) = ↓
π((1, 0)) = ↑
	DEBUG: pi(action=↑, state=(1, 1)): Got 2 best actions: ↑, ←, choosing ↑
π((1, 1)) = ↑
	DEBUG: pi(action=↓, state=(1, 2)): Got 2 best actions: ↓, ←, choosing ↓
π((1, 2)) = ↓
π((1, 3)) = ↓
π((2, 0)) = ↑
	DEBUG: pi(action=↑, state=(2, 1)): Got 2 best actions: ↑, →, choosing ↑
π((2, 1)) = ↑
	DEBUG: pi(action=↓, state=(2, 2)): Got 2 best actions: ↓, →, choosing ↓
π((2, 2)) = ↓
π((2, 3)) = ↓
	DEBUG: pi(action=↑, state=(3, 0)): Got 2 best actions: ↑, →, choosing ↑
π((3, 0)) = ↑
π((3, 1)) = →
π((3, 2)) = →


In [11]:
def optimal_policy(mdp: SmallGridWorld) -> tuple[Grid, Policy]:
    v = Grid(4)
    pi = generate_uniform_random_policy(mdp)
    v_pi = calc_svals(mdp, pi)

    while not v_pi.close(v):
        v.copy(v_pi)
        pi = generate_greedy_policy(mdp, v)
        v_pi = calc_svals(mdp, pi)

    return v_pi, pi

In [12]:
v, pi = optimal_policy(small_grid)

Converged after 159 iterations.
Converged after 4 iterations.
Converged after 4 iterations.


In [13]:
v

array([[ 0., -1., -2., -3.],
       [-1., -2., -3., -2.],
       [-2., -3., -2., -1.],
       [-3., -2., -1.,  0.]], dtype=float32)

In [14]:
for s in small_grid.states():
    for a in small_grid.actions():
        prob = pi(a, s, dbg=True)
        if prob == 1.0:
            print(f"π({s}) = {a}")

π((0, 1)) = ←
π((0, 2)) = ←
	DEBUG: pi(action=↓, state=(0, 3)): Got 2 best actions: ↓, ←, choosing ↓
π((0, 3)) = ↓
π((1, 0)) = ↑
	DEBUG: pi(action=↑, state=(1, 1)): Got 2 best actions: ↑, ←, choosing ↑
π((1, 1)) = ↑
	DEBUG: pi(action=↑, state=(1, 2)): Got 4 best actions: ↑, ↓, ←, →, choosing ↑
π((1, 2)) = ↑
π((1, 3)) = ↓
π((2, 0)) = ↑
	DEBUG: pi(action=↑, state=(2, 1)): Got 4 best actions: ↑, ↓, ←, →, choosing ↑
π((2, 1)) = ↑
	DEBUG: pi(action=↓, state=(2, 2)): Got 2 best actions: ↓, →, choosing ↓
π((2, 2)) = ↓
π((2, 3)) = ↓
	DEBUG: pi(action=↑, state=(3, 0)): Got 2 best actions: ↑, →, choosing ↑
π((3, 0)) = ↑
π((3, 1)) = →
π((3, 2)) = →


## Value Iteration

Instead of going iterating through evaluating policy and then improving it, I can directly find the optimal state values and therefore the optimal policy using the Belman's optimality equation.

$$
\begin{align}
v_*(s) &= \underset{a}{max} \left( r(s, a) + \gamma E_P \left[ v_*(s') \right] \right) \\
&= \underset{a}{max} \left( r(s, a) + \gamma \sum_{s' \in S} P(s' \vert s, a) \; v_*(s') \right)
\end{align}
$$

Using dynamic programming I can iteratively find the solution -
$$
v^*_{k+1}(s) = \underset{a}{max} \left( r(s, a) + \gamma \sum_{s' \in S} P(s' \vert s, a) \; v^*_k(s') \right)
$$

In [15]:
def optimal_svals(mdp: SmallGridWorld) -> Grid:
    v = Grid(4, random=True)
    v_next = Grid(4)
    i = 0
    while i < MAX_ITERS and not v.close(v_next):
        v.copy(v_next)
        v_next.clear()
        q = partial(qval, mdp, v)
        for s in mdp.states():
            v_next[s] = max(q(s, a) for a in mdp.actions())
        i += 1
    print(f"Converged after {i} iterations.")
    v.round()
    return v

In [16]:
v_star = optimal_svals(small_grid)
v_star

Converged after 4 iterations.


array([[ 0., -1., -2., -3.],
       [-1., -2., -3., -2.],
       [-2., -3., -2., -1.],
       [-3., -2., -1.,  0.]], dtype=float32)

In [17]:
pi_star = generate_greedy_policy(small_grid, v_star)
for s in small_grid.states():
    for a in small_grid.actions():
        prob = pi_star(a, s, dbg=True)
        if prob == 1.0:
            print(f"π*({s}) = {a}")

π*((0, 1)) = ←
π*((0, 2)) = ←
	DEBUG: pi(action=↓, state=(0, 3)): Got 2 best actions: ↓, ←, choosing ↓
π*((0, 3)) = ↓
π*((1, 0)) = ↑
	DEBUG: pi(action=↑, state=(1, 1)): Got 2 best actions: ↑, ←, choosing ↑
π*((1, 1)) = ↑
	DEBUG: pi(action=↑, state=(1, 2)): Got 4 best actions: ↑, ↓, ←, →, choosing ↑
π*((1, 2)) = ↑
π*((1, 3)) = ↓
π*((2, 0)) = ↑
	DEBUG: pi(action=↑, state=(2, 1)): Got 4 best actions: ↑, ↓, ←, →, choosing ↑
π*((2, 1)) = ↑
	DEBUG: pi(action=↓, state=(2, 2)): Got 2 best actions: ↓, →, choosing ↓
π*((2, 2)) = ↓
π*((2, 3)) = ↓
	DEBUG: pi(action=↑, state=(3, 0)): Got 2 best actions: ↑, →, choosing ↑
π*((3, 0)) = ↑
π*((3, 1)) = →
π*((3, 2)) = →
